In [2]:
from typing import Optional
from torch import nn
from torch.nn import functional as F
import torch


class SpaceTranslator(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_layers, activation, dropout_rate):
        super().__init__()
        layers = []
        last = input_dim
        
        for hidden in hidden_layers:
            layers += [nn.Linear(last, hidden), activation(), nn.LayerNorm(hidden), nn.Dropout(dropout_rate)]
            last = hidden
            
        layers.append(nn.Linear(last, output_dim))
        self.net = nn.Sequential(*layers)

        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1/0.07))
        
        self.apply(self.init_weights)
        
    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)
        elif isinstance(module, nn.LayerNorm):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(self, x):
        return self.net(x)

In [3]:
from pathlib import Path
import numpy as np
import torch
import pandas as pd

'''Code from https://github.com/Mamiglia/challenge'''

def mrr(pred_indices: np.ndarray, gt_indices: np.ndarray) -> float:
    """
    Compute Mean Reciprocal Rank (MRR)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries (top-K)
        gt_indices: (N,) array of ground truth indices
    Returns:
        mrr: Mean Reciprocal Rank
    """
    reciprocal_ranks = []
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i] == gt_indices[i])[0]
        if matches.size > 0:
            reciprocal_ranks.append(1.0 / (matches[0] + 1))
        else:
            reciprocal_ranks.append(0.0)
    return np.mean(reciprocal_ranks)


def recall_at_k(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int) -> float:
    """Compute Recall@k
    Args:
        pred_indices: (N, N) array of top indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        recall: Recall@k
    """
    recall = 0
    for i in range(len(gt_indices)):
        if gt_indices[i] in pred_indices[i, :k]:
            recall += 1
    recall /= len(gt_indices)
    return recall

import numpy as np

def ndcg(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int = 100) -> float:
    """
    Compute Normalized Discounted Cumulative Gain (NDCG@k)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        ndcg: NDCG@k
    """
    ndcg_total = 0.0
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i, :k] == gt_indices[i])[0]
        if matches.size > 0:
            rank = matches[0] + 1
            ndcg_total += 1.0 / np.log2(rank + 1)  # DCG (IDCG = 1)
    return ndcg_total / len(gt_indices)



@torch.inference_mode()
def evaluate_retrieval(translated_embd, image_embd, gt_indices, max_indices = 99, batch_size=100):
    """Evaluate retrieval performance using cosine similarity
    Args:
        translated_embd: (N_captions, D) translated caption embeddings
        image_embd: (N_images, D) image embeddings
        gt_indices: (N_captions,) ground truth image indices for each caption
        max_indices: number of top predictions to consider
    Returns:
        results: dict of evaluation metrics
    
    """
    # Compute similarity matrix
    if isinstance(translated_embd, np.ndarray):
        translated_embd = torch.from_numpy(translated_embd).float()
    if isinstance(image_embd, np.ndarray):
        image_embd = torch.from_numpy(image_embd).float()
    
    n_queries = translated_embd.shape[0]
    device = translated_embd.device
    
    # Prepare containers for the fragments to be reassembled
    all_sorted_indices = []
    l2_distances = []
    
    # Process in batches - the narrow gate approach
    for start_idx in range(0, n_queries, batch_size):
        batch_slice = slice(start_idx, min(start_idx + batch_size, n_queries))
        batch_translated = translated_embd[batch_slice]
        batch_img_embd = image_embd[batch_slice]
        
        # Compute similarity only for this batch
        batch_similarity = batch_translated @ batch_img_embd.T

        # Get top-k predictions for this batch
        batch_indices = batch_similarity.topk(k=max_indices, dim=1, sorted=True).indices.numpy()
        all_sorted_indices.append(gt_indices[batch_slice][batch_indices])

        # Compute L2 distance for this batch
        batch_gt = gt_indices[batch_slice]
        batch_gt_embeddings = image_embd[batch_gt]
        batch_l2 = (batch_translated - batch_gt_embeddings).norm(dim=1)
        l2_distances.append(batch_l2)
    
    # Reassemble the fragments
    sorted_indices = np.concatenate(all_sorted_indices, axis=0)
    
    # Apply the sacred metrics to the whole
    metrics = {
        'mrr': mrr,
        'ndcg': ndcg,
        'recall_at_1': lambda preds, gt: recall_at_k(preds, gt, 1),
        'recall_at_3': lambda preds, gt: recall_at_k(preds, gt, 3),
        'recall_at_5': lambda preds, gt: recall_at_k(preds, gt, 5),
        'recall_at_10': lambda preds, gt: recall_at_k(preds, gt, 10),
        'recall_at_50': lambda preds, gt: recall_at_k(preds, gt, 50),
    }
    
    results = {
        name: func(sorted_indices, gt_indices)
        for name, func in metrics.items()
    }
    
    l2_dist = torch.cat(l2_distances, dim=0).mean().item()
    results['l2_dist'] = l2_dist
    
    return results

def eval_on_val(x_val: np.ndarray, y_val: np.ndarray, model: nn.Module, device, scale=None) -> dict:
    gt_indices = torch.arange(len(y_val))
    
    model.eval()

    with torch.inference_mode():
        translated = model(x_val.to(device)).to('cpu')

        if scale is not None:
            translated *= scale

    results = evaluate_retrieval(translated, y_val, gt_indices)
    
    return results


def generate_submission(model: nn.Module, test_path: Path, output_file="submission-dirmodel.csv", device=None):
    test_data = np.load(test_path)
    sample_ids = test_data['captions/ids']
    test_embds = test_data['captions/embeddings']
    test_embds = torch.from_numpy(test_embds).float()

    with torch.no_grad():
        pred_embds = model(test_embds.to(device)).cpu()

    print("Generating submission file...")

    if isinstance(pred_embds, torch.Tensor):
        pred_embds = pred_embds.cpu().numpy()

    df_submission = pd.DataFrame({'id': sample_ids, 'embedding': pred_embds.tolist()})

    df_submission.to_csv(output_file, index=False, float_format='%.17g')
    print(f"✓ Saved submission to {output_file}")

    return df_submission

In [4]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pathlib import Path
from tqdm import tqdm

def info_nce_loss(dir_preds, img_targets, logit_scale):
    # dir_preds = F.normalize(dir_preds, dim=-1)
    # img_targets = F.normalize(img_targets, dim=-1)

    # mult, not div
    logits = (dir_preds @ img_targets.T) * logit_scale.exp()

    labels = torch.arange(logits.size(0), device=logits.device)

    loss_t2i = F.cross_entropy(logits, labels)
    loss_i2t = F.cross_entropy(logits.T, labels)

    return 0.5 * (loss_t2i + loss_i2t)

def train_model_direction(
    model: SpaceTranslator,
    model_path: Path,
    train_dataset: TensorDataset,
    val_dataset: TensorDataset,
    batch_size: int,
    epochs: int,
    lr: float,
    patience: int
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print(f"🚀 Using device: {device}")

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.5,           # reduce LR by half when plateau
        patience=2,           # wait 2 epochs before reducing LR
        min_lr=1e-6,          # don't go below this LR
        verbose=True
    )

    best_val_loss = float('inf')
    no_improvements = 0

    for epoch in range(1, epochs + 1):
        model.train()
        running_train_loss = 0.0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
        for X_batch, y_batch in progress_bar:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            # X_batch = F.normalize(X_batch, dim=-1)
            # y_batch = F.normalize(y_batch, dim=-1)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = info_nce_loss(outputs, y_batch, model.logit_scale)

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        train_loss = running_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        running_val_loss = 0.0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                # X_batch = F.normalize(X_batch, dim=-1)
                # y_batch = F.normalize(y_batch, dim=-1)

                outputs = model(X_batch)
                loss = info_nce_loss(outputs, y_batch, model.logit_scale)
                running_val_loss += loss.item()

        val_loss = running_val_loss / len(val_loader)

        # Step the scheduler
        scheduler.step(val_loss)

        print(f"📘 Epoch {epoch:03d} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | LR: {optimizer.param_groups[0]['lr']:.2e}")

        # Optional: external validation/test
        test(val_dataset, model, device)

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvements = 0

            Path(model_path).parent.mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), model_path)
            print(f"💾 Saved new best model (val_loss={val_loss:.6f})")
        else:
            no_improvements += 1
            if no_improvements >= patience:
                print("⏹ Early stopping triggered.")
                break

    print(f"✅ Training complete. Best Val Loss: {best_val_loss:.6f}")

    return model


def get_data(data_path: Path):
    data = np.load(data_path)
    caption_embeddings = data['captions/embeddings']
    image_embeddings = data['images/embeddings']
    caption_labels = data['captions/label']
    data.close()

    X_abs, y_abs = torch.tensor(caption_embeddings), torch.tensor(image_embeddings[np.argmax(caption_labels, axis=1)])

    return X_abs, y_abs
    
def get_datasets(X_abs, y_abs) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:    
    print('Texts shape', X_abs.shape)
    print('Images shape', y_abs.shape)
    
    dataset = TensorDataset(X_abs, y_abs)
    train_dataset, val_dataset = random_split(dataset, [0.9, 0.1], generator=torch.Generator().manual_seed(42))
    
    return train_dataset, val_dataset

def test(val_dataset: TensorDataset, model: nn.Module, device, scale=None):
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset))
    for x_val, y_val in val_loader:
        results = eval_on_val(x_val, y_val, model=model, device=device, scale=scale)
    return results

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = 1024
output_dim = 1536
hidden_layers=[1256, 2048]
dropout_rate = 0.3

batch_size= 512
lr= 0.0005 # 0.0006
epochs= 200
patience = 5

data_path= '/kaggle/input/aml-competition/train/train/train.npz'
test_path= '/kaggle/input/aml-competition/test/test/test.clean.npz'

In [8]:
dir_x, dir_y = get_data(data_path)

target_norm_mean = dir_y.norm(dim=1).mean().item()

dir_train_dataset, dir_val_dataset = get_datasets(dir_x, dir_y)

print('Target norm mean', target_norm_mean)

Texts shape torch.Size([125000, 1024])
Images shape torch.Size([125000, 1536])
Target norm mean 25.93919563293457


In [18]:
model_args = {
    'input_dim': input_dim,
    'output_dim': output_dim,
    'hidden_layers': hidden_layers,
    'dropout_rate': dropout_rate,
    'activation': nn.GELU()
}

dir_model = SpaceTranslator(**model_args).to(device)

train_model_direction(dir_model, './models/dir-model.pth', dir_train_dataset, dir_val_dataset, batch_size, epochs, lr, patience)

print('Finished training. Now testing using best model...')

state = torch.load('./models/dir-model.pth')
dir_model.load_state_dict(state)
results = test(dir_val_dataset, dir_model, device)
print("Test Results:", results)

🚀 Using device: cuda


📘 Epoch 001 | Train Loss: 318.539068 | Val Loss: 81.438514 | LR: 5.00e-04
💾 Saved new best model (val_loss=81.438514)


📘 Epoch 002 | Train Loss: 99.156497 | Val Loss: 29.774160 | LR: 5.00e-04
💾 Saved new best model (val_loss=29.774160)


📘 Epoch 003 | Train Loss: 41.014556 | Val Loss: 13.851297 | LR: 5.00e-04
💾 Saved new best model (val_loss=13.851297)


📘 Epoch 004 | Train Loss: 22.011934 | Val Loss: 8.558918 | LR: 5.00e-04
💾 Saved new best model (val_loss=8.558918)


...

📘 Epoch 060 | Train Loss: 0.187490 | Val Loss: 0.970065 | LR: 3.13e-05


📘 Epoch 061 | Train Loss: 0.183968 | Val Loss: 0.970354 | LR: 3.13e-05


📘 Epoch 062 | Train Loss: 0.184603 | Val Loss: 0.970935 | LR: 1.56e-05


📘 Epoch 063 | Train Loss: 0.178825 | Val Loss: 0.961557 | LR: 1.56e-05


📘 Epoch 064 | Train Loss: 0.174347 | Val Loss: 0.963780 | LR: 1.56e-05
⏹ Early stopping triggered.
✅ Training complete. Best Val Loss: 0.960105
Finished training. Now testing using best model...
Test Results: {'mrr': 0.9385330845008674, 'ndcg': 0.9534359349718273, 'recall_at_1': 0.90184, 'recall_at_3': 0.97168, 'recall_at_5': 0.98296, 'recall_at_10': 0.99272, 'recall_at_50': 0.99912, 'l2_dist': 25.877605438232422}


In [20]:
generate_submission(dir_model, Path(test_path), output_file="dav.csv", device=device)

Generating submission file...
✓ Saved submission to dav.csv


,id,embedding
0,1,"[0.027106191962957382, 0.006234061438590288, 0..."
1,2,"[-0.0018218876793980598, 0.010847451165318489,..."
2,3,"[0.006840117275714874, 0.01300758309662342, 0...."
3,4,"[0.06066161394119263, -0.03855207562446594, -0..."
4,5,"[0.05592562258243561, 0.02765999175608158, -0...."
...,...,...
1495,1496,"[-0.01505435723811388, 0.007482768967747688, 0..."
1496,1497,"[0.03140179440379143, 0.03376109153032303, 0.0..."
1497,1498,"[0.06372750550508499, -0.00012322794646024704,..."
1498,1499,"[-0.008761415258049965, 0.02096746675670147, 0..."


In [19]:
import optuna
from optuna.pruners import MedianPruner

ACTIVATIONS = {
    #"relu": nn.ReLU,
    "gelu": nn.GELU,
    "silu": nn.SiLU,
    'selu': nn.SELU,
    'celu': nn.CELU
    #"leakyrelu": nn.LeakyReLU
}

def objective(trial, train_dataset, val_dataset, epochs: int = 10, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    n_layers = trial.suggest_int("n_layers", 1, 4)
    layer_choices = [1024, 1536, 2048, 4096]
    hidden_layers = [trial.suggest_categorical(f"n_units_l{i}", layer_choices) for i in range(n_layers)]

    activation_name = trial.suggest_categorical("activation", list(ACTIVATIONS.keys()))
    activation_fn = ACTIVATIONS[activation_name]
    
    batch_size = trial.suggest_categorical("batch_size", [256, 512, 1024, 2048, 4096])
    lr = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
    dropout_rate = trial.suggest_categorical('dropout_rate', [0.1, 0.2, 0.3])

    weight_decay = trial.suggest_float("weight_decay", 1e-4, 1e-2, log=True)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = SpaceTranslator(
        input_dim=1024, 
        output_dim=1536,
        hidden_layers=hidden_layers,
        activation=activation_fn,
        dropout_rate=dropout_rate
    )
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.5,           # reduce LR by half when plateau
        patience=2,           # wait 2 epochs before reducing LR
        min_lr=1e-6,          # don't go below this LR
        verbose=True
    )

    best_val_loss = float('inf')
    no_improvements = 0

    for epoch in range(1, epochs + 1):
        model.train()
        running_train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = info_nce_loss(outputs, y_batch, model.logit_scale)

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        train_loss = running_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        running_val_loss = 0.0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                outputs = model(X_batch)
                loss = info_nce_loss(outputs, y_batch, model.logit_scale)
                running_val_loss += loss.item()

        val_loss = running_val_loss / len(val_loader)

        scheduler.step(val_loss)

        results = test(val_dataset, model, device)

        trial.report(results['mrr'], epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()


    return results['mrr']


def run_optuna_search(data_path: Path, n_trials: int = 30, epochs: int = 10, n_jobs: int = 1, pruner=None):
    if pruner is None:
        pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=1)

    X, Y = get_data(data_path)    
    train_dataset, val_dataset = get_datasets(X, Y)

    study = optuna.create_study(direction="maximize", pruner=pruner)
    func = lambda trial: objective(trial, train_dataset=train_dataset, val_dataset=val_dataset,
                                   epochs=epochs)
    study.optimize(func, n_trials=n_trials, n_jobs=n_jobs)

    print("Study statistics:")
    print("  Number of finished trials: ", len(study.trials))
    print("  Best trial:")
    trial = study.best_trial
    print("    Value: ", trial.value)
    print("    Params: ")
    for k, v in trial.params.items():
        print(f"      {k}: {v}")

    return study

In [ ]:
study = run_optuna_search(data_path=data_path, n_trials=150, epochs=40, n_jobs=1)
study.trials_dataframe().to_csv("optuna_trials.csv", index=False)

best_trial_number = study.best_trial.number
print("Best params:", study.best_params)
print("Best trial number:", study.best_trial.number)

[I 2025-11-04 12:20:12,966] A new study created in memory with name: no-name-60b28e84-1ada-4a77-ad28-8a3fc859e01a


Texts shape torch.Size([125000, 1024])
Images shape torch.Size([125000, 1536])


[I 2025-11-04 12:24:37,712] Trial 0 finished with value: 0.9028130716112096 and parameters: {'n_layers': 2, 'n_units_l0': 4096, 'n_units_l1': 1536, 'activation': 'selu', 'batch_size': 512, 'lr': 7.50332230817106e-05, 'dropout_rate': 0.2, 'weight_decay': 0.00013430021043607675}. Best is trial 0 with value: 0.9028130716112096.
[I 2025-11-04 12:27:44,752] Trial 1 finished with value: 0.9047564361940321 and parameters: {'n_layers': 1, 'n_units_l0': 1536, 'activation': 'selu', 'batch_size': 256, 'lr': 0.00013520855942195204, 'dropout_rate': 0.1, 'weight_decay': 0.004668758209667024}. Best is trial 1 with value: 0.9047564361940321.
[I 2025-11-04 12:31:21,241] Trial 2 finished with value: 0.9182350042165024 and parameters: {'n_layers': 2, 'n_units_l0': 1024, 'n_units_l1': 2048, 'activation': 'silu', 'batch_size': 256, 'lr': 2.8966031405401815e-05, 'dropout_rate': 0.3, 'weight_decay': 0.00015524028860226148}. Best is trial 2 with value: 0.9182350042165024.
[I 2025-11-04 12:35:00,329] Trial 3 f